### 시군구별 세대수
+ KOSIS 공유 서비스: https://kosis.kr/openapi/index/index.jsp
### 동별 세대수
+ 공공데이터 포털: 행정안전부_법정동별(행정동 통반단위) 주민등록 인구 및 세대현황 https://www.data.go.kr/data/15108071/openapi.do

<b>2023년 5월</b>부터 **시군구별 세대수** 데이터를 수집

# 1. 시군구별 행정동(법정동) 코드 추출
아래의 표와 같이 구하고자 하는 지역 코드를 확인해야 합니다.  
해당 값은 법정동/행정동 코드의 앞 **5자리**만 사용합니다. (시,군,구 단위 데이터 수집)  
(칠곡군 동명면의 경우 해당 API를 통한 데이터 수집이 불가능합니다.) >> 데이터가 없습니다.

|소재|C1_NM|C1|
|:------:|:---:|:---:|
|대구광역시|중구|27110|
|대구광역시|동구|27140|
|대구광역시|서구|27170|
|대구광역시|남구|27200|
|대구광역시|북구|27230|
|대구광역시|수성구|27260|
|대구광역시|달서구|27290|
|대구광역시|달성군|27710|
|경상북도|경산시|47290|
|경상북도|고령군|47830|

In [1]:
#라이브러리 설치

import pandas as pd
# from pandas.io.json import json_normalize
import requests
# import os
from tqdm.notebook import tqdm
# import time
# import pprint
import json
import pymysql
# import csv
from urllib.parse import quote
from sqlalchemy import create_engine
import sqlalchemy
from datetime import datetime
# from datetime import timedelta
from dateutil.relativedelta import relativedelta

In [2]:
#지역코드 삽입
daegu_list = '27110+27140+27170+27200+27230+27260+27290+27710' #대구 시군구 지역코드
gyeongbuk_list= '47290+47830' #경북 경산시, 고령군 지역코드
target_list = daegu_list + '+' + gyeongbuk_list

# 2. 통계청 공유서비스에서 API 주소 추출
해당 데이터는  
https://kosis.kr/openapi/index/index.jsp   [KOSIS 공유서비스]  
에서 수집 가능합니다.  
해당 데이터는 **[개발가이드] - [통계자료] - [URL 생성] - [통계표명: 주민등록세대수]** 검색 시 확인 가능합니다.
    (https://kosis.kr/openapi/devGuide/devGuide_0203List.jsp)

In [61]:
# now = datetime.now()
# print("사용시간 :" , now)
# before_1_months = now + relativedelta(months=-1)
# print("1달 전 :", before_1_months)#한달 전
# before_month=before_1_months.strftime("%Y%m")
# print("1달전 년월 : ",before_month)
month = '202306'
# start_date = str(month)  #데이터 수집 시작 날짜(년월)
# end_date = str(month)    #데이터 수집 끝 날짜(년월)
# start_date = str(start_date) #형식변경
# end_date = str(end_date)#형식변경
apiKey= 'YTg2NjFjM2FkOGFjZWQ5ODBmMDY1M2IwZTZiNjRlNzg' #통계청 API 코드

In [62]:
#API 주소 입력
json_URL='https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey='+apiKey+'=&itmId=T1+&objL1='+target_list+'&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe='+month+'&endPrdDe='+month+'&orgId=101&tblId=DT_1B040B3'
api_get = requests.get(json_URL,verify=False)
api_get #제대로 받아오면 <Response [200]>가 출력됨

d:\PythonProject\data-gatherer\data-gatherer-venv\lib\site-packages\urllib3\connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'kosis.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

# 3. 시군구별 세대수 데이터 추출 및 전처리

시군구별 데이터를 추출하고 필요한 부분만 사용하기 때문에 전처리를 진행

In [63]:
api_get_data = api_get.json() #json 형식으로 받아옴
api_get_data 

[{'TBL_NM': '행정구역(시군구)별 주민등록세대수',
  'PRD_DE': '202306',
  'TBL_ID': 'DT_1B040B3',
  'ITM_NM': '세대수',
  'ITM_NM_ENG': 'Household',
  'ITM_ID': 'T1',
  'UNIT_NM': '호',
  'ORG_ID': '101',
  'UNIT_NM_ENG': 'Households',
  'C1_OBJ_NM': '행정구역(시군구)별',
  'C1_OBJ_NM_ENG': 'By Administrative District',
  'DT': '42582',
  'PRD_SE': 'M',
  'C1': '27110',
  'C1_NM': '중구',
  'C1_NM_ENG': 'Jung-gu'},
 {'TBL_NM': '행정구역(시군구)별 주민등록세대수',
  'PRD_DE': '202306',
  'TBL_ID': 'DT_1B040B3',
  'ITM_NM': '세대수',
  'ITM_NM_ENG': 'Household',
  'ITM_ID': 'T1',
  'UNIT_NM': '호',
  'ORG_ID': '101',
  'UNIT_NM_ENG': 'Households',
  'C1_OBJ_NM': '행정구역(시군구)별',
  'C1_OBJ_NM_ENG': 'By Administrative District',
  'DT': '160720',
  'PRD_SE': 'M',
  'C1': '27140',
  'C1_NM': '동구',
  'C1_NM_ENG': 'Dong-gu'},
 {'TBL_NM': '행정구역(시군구)별 주민등록세대수',
  'PRD_DE': '202306',
  'TBL_ID': 'DT_1B040B3',
  'ITM_NM': '세대수',
  'ITM_NM_ENG': 'Household',
  'ITM_ID': 'T1',
  'UNIT_NM': '호',
  'ORG_ID': '101',
  'UNIT_NM_ENG': 'Households',
  'C1

In [64]:
df = pd.json_normalize(api_get_data) #데이터 프레임 형식으로 불러옴
print(df.head(3))
print(df.tail(3))

               TBL_NM  PRD_DE      TBL_ID ITM_NM ITM_NM_ENG ITM_ID UNIT_NM  \
0  행정구역(시군구)별 주민등록세대수  202306  DT_1B040B3    세대수  Household     T1       호   
1  행정구역(시군구)별 주민등록세대수  202306  DT_1B040B3    세대수  Household     T1       호   
2  행정구역(시군구)별 주민등록세대수  202306  DT_1B040B3    세대수  Household     T1       호   

  ORG_ID UNIT_NM_ENG   C1_OBJ_NM               C1_OBJ_NM_ENG      DT PRD_SE  \
0    101  Households  행정구역(시군구)별  By Administrative District   42582      M   
1    101  Households  행정구역(시군구)별  By Administrative District  160720      M   
2    101  Households  행정구역(시군구)별  By Administrative District   81819      M   

      C1 C1_NM C1_NM_ENG  
0  27110    중구   Jung-gu  
1  27140    동구   Dong-gu  
2  27170    서구    Seo-gu  
               TBL_NM  PRD_DE      TBL_ID ITM_NM ITM_NM_ENG ITM_ID UNIT_NM  \
7  행정구역(시군구)별 주민등록세대수  202306  DT_1B040B3    세대수  Household     T1       호   
8  행정구역(시군구)별 주민등록세대수  202306  DT_1B040B3    세대수  Household     T1       호   
9  행정구역(시군구)별 주민등록세대수  20230

In [65]:
df = df[['PRD_DE','C1_NM','C1','DT']] #수집날짜, 시도코드, 시도, 세대수
df.rename(columns = {'PRD_DE':'통계년월','C1_NM':'시군구명','C1':'행정동코드','DT':'세대수'},inplace=True) #법정동코드, 행정동코드 상관x
df

,통계년월,시군구명,행정동코드,세대수
0,202306,중구,27110,42582
1,202306,동구,27140,160720
2,202306,서구,27170,81819
3,202306,남구,27200,76592
4,202306,북구,27230,192580
5,202306,수성구,27260,171299
6,202306,달서구,27290,236791
7,202306,달성군,27710,113996
8,202306,경산시,47290,129053
9,202306,고령군,47830,16906


# 4. API 주소를 활용한 동명면 데이터 추출

In [66]:
def name_sort_change(df):
    #이름 변경
    df.rename(columns={'hhCnt':'세대수','tong':'통','femlNmprCnt':'여자인구수','stdgCd':'법정동코드','maleFemlRate':'남여비율',
                       'stdgNm':'법정동명','ban':'반','totNmprCnt':'총인구수','ctpvNm':'시도명','maleNmprCnt':'남자인구수',
                       'sggNm':'시군구명','dongNm':'행정동명','hhNmpr':'세대당인구','admmCd':'행정기관코드',
                       'statsYm':'통계년월','liNm':'리명'},inplace=True)
    #기존의 표와 같이 순서 변경
    df = df[['통계년월','법정동코드','시도명','시군구명','법정동명','리명','행정기관코드','행정동명','통',
             '반','총인구수','세대수','세대당인구','남자인구수','여자인구수','남여비율']]
    return df

In [67]:
service_url = 'http://apis.data.go.kr/1741000/stdgPpltnHhStus'
# 일반 인증키(Encoding)	
api_key = 'IVgu%2FZBjA6hpLryyEOpySC2RhogOhaJIUqlXN8Uyj3Gxw4s3dX0qMxfgXMTLl60%2Fs2EYAMUsyyzTqwVOnjoIhg%3D%3D'
# 일반 인증키(Decoding)	
# api_key = 'IVgu/ZBjA6hpLryyEOpySC2RhogOhaJIUqlXN8Uyj3Gxw4s3dX0qMxfgXMTLl60/s2EYAMUsyyzTqwVOnjoIhg=='
region_code = '4785032000'    # 칠곡군 동명면 지역코드
# 조회결과구분. 광역시도 단위 : 1, 시군구 단위 : 2, 읍면동 단위 : 3, 읍면동 통반 단위 : 4(기본값 : 4)
level = '4'
# 등록구분. 전체:1, 거주자:2, 거주불명자:3, 재외국민:4(기본값 : 1)
reg_code = '1'
# 타입. XML, JSON(기본값 : XML)
req_type = 'json'
# 페이지 크기. 페이지당 목록 수(1~100)(기본값 : 10)
pageRows = '100'
# 페이지 번호. 기본값 : 1
pageNo = '1'


url = f'{service_url}/selectStdgPpltnHhStus?serviceKey={api_key}&stdgCd={region_code}&srchFrYm={month}&srchToYm={month}&lv={level}&regSeCd={reg_code}&type={req_type}&numOfRows={pageRows}&pageNo={pageNo}'
print(url)

http://apis.data.go.kr/1741000/stdgPpltnHhStus/selectStdgPpltnHhStus?serviceKey=IVgu%2FZBjA6hpLryyEOpySC2RhogOhaJIUqlXN8Uyj3Gxw4s3dX0qMxfgXMTLl60%2Fs2EYAMUsyyzTqwVOnjoIhg%3D%3D&stdgCd=4785032000&srchFrYm=202306&srchToYm=202306&lv=4&regSeCd=1&type=json&numOfRows=100&pageNo=1


In [68]:
response = requests.get(url,verify=False)
contents = response.text
json_ob = json.loads(contents)
body = json_ob['Response']['items']['item']
df_dongmyeong = pd.json_normalize(body)

In [69]:
df_dongmyeong

,hhCnt,tong,femlNmprCnt,stdgCd,maleFemlRate,stdgNm,ban,liNm,totNmprCnt,ctpvNm,maleNmprCnt,sggNm,dongNm,hhNmpr,admmCd,statsYm
0,25,1,19,4785032030,0.68,동명면,1,금암리,32,경상북도,13,칠곡군,동명면,1.28,4785032000,202306
1,6,1,3,4785032030,2.33,동명면,2,금암리,10,경상북도,7,칠곡군,동명면,1.67,4785032000,202306
2,99,1,89,4785032030,0.87,동명면,3,금암리,166,경상북도,77,칠곡군,동명면,1.68,4785032000,202306
3,14,1,9,4785032030,1.22,동명면,4,금암리,20,경상북도,11,칠곡군,동명면,1.43,4785032000,202306
4,35,1,30,4785032030,0.97,동명면,5,금암리,59,경상북도,29,칠곡군,동명면,1.69,4785032000,202306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,18,17,18,4785032038,1.11,동명면,3,봉암리,38,경상북도,20,칠곡군,동명면,2.11,4785032000,202306
61,27,17,17,4785032038,1.53,동명면,4,봉암리,43,경상북도,26,칠곡군,동명면,1.59,4785032000,202306
62,12,17,10,4785032038,1.30,동명면,5,봉암리,23,경상북도,13,칠곡군,동명면,1.92,4785032000,202306
63,70,17,46,4785032038,1.48,동명면,6,봉암리,114,경상북도,68,칠곡군,동명면,1.63,4785032000,202306


In [70]:
df_dongmyeong = name_sort_change(df_dongmyeong) #컬럼 명 변경 및 순서 변경

In [71]:

df_dongmyeong['세대수'] = df_dongmyeong['세대수'].apply(int) #세대수를 합쳐주기 위해서 int로 변경
df_dongmyeong

,통계년월,법정동코드,시도명,시군구명,법정동명,리명,행정기관코드,행정동명,통,반,총인구수,세대수,세대당인구,남자인구수,여자인구수,남여비율
0,202306,4785032030,경상북도,칠곡군,동명면,금암리,4785032000,동명면,1,1,32,25,1.28,13,19,0.68
1,202306,4785032030,경상북도,칠곡군,동명면,금암리,4785032000,동명면,1,2,10,6,1.67,7,3,2.33
2,202306,4785032030,경상북도,칠곡군,동명면,금암리,4785032000,동명면,1,3,166,99,1.68,77,89,0.87
3,202306,4785032030,경상북도,칠곡군,동명면,금암리,4785032000,동명면,1,4,20,14,1.43,11,9,1.22
4,202306,4785032030,경상북도,칠곡군,동명면,금암리,4785032000,동명면,1,5,59,35,1.69,29,30,0.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,202306,4785032038,경상북도,칠곡군,동명면,봉암리,4785032000,동명면,17,3,38,18,2.11,20,18,1.11
61,202306,4785032038,경상북도,칠곡군,동명면,봉암리,4785032000,동명면,17,4,43,27,1.59,26,17,1.53
62,202306,4785032038,경상북도,칠곡군,동명면,봉암리,4785032000,동명면,17,5,23,12,1.92,13,10,1.30
63,202306,4785032038,경상북도,칠곡군,동명면,봉암리,4785032000,동명면,17,6,114,70,1.63,68,46,1.48


In [72]:
df_dongmyeong['행정동코드'] = '4785032000' #행정동코드가 여러개가 있는데, 여기서 하나로 통일 (groupby를 위해서)
df_dongmyeong = df_dongmyeong.groupby(['통계년월','행정동명','행정동코드']).sum() #groupby
df_dongmyeong.reset_index(inplace=True) #인덱스 초기화
df_dongmyeong = df_dongmyeong[['통계년월','행정동명','행정동코드','세대수']] #필요 부분만 추출
df_dongmyeong.rename(columns={'행정동명':'시군구명'},inplace=True) #컬럼명 변경
df_dongmyeong

,통계년월,시군구명,행정동코드,세대수
0,202306,동명면,4785032000,2969


In [73]:
df= pd.concat([df,df_dongmyeong],axis = 0) #데이터 결합
df.drop_duplicates(keep = 'first',inplace=True) #중복값 제거 >> 중복값이 있을 때 첫번째 값만 남겨둠
df.reset_index(inplace=True,drop=True) #인덱스 초기화
df = df.replace('동명면', '칠곡군') #동명면을 칠곡군으로 변경 >> 시군구 단위는 동명면이 아닌 칠곡군이기 대문
df

,통계년월,시군구명,행정동코드,세대수
0,202306,중구,27110,42582
1,202306,동구,27140,160720
2,202306,서구,27170,81819
3,202306,남구,27200,76592
4,202306,북구,27230,192580
5,202306,수성구,27260,171299
6,202306,달서구,27290,236791
7,202306,달성군,27710,113996
8,202306,경산시,47290,129053
9,202306,고령군,47830,16906


# 5. 세대수 데이터 DB에 저장

In [74]:
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

In [75]:
conn = dbconnect.db_connect("DEMO_DW")
cursor = conn.cursor()

query = f'SELECT EXISTS (SELECT * FROM household_gu WHERE 통계년월 = {month})'

cursor.execute(query)
row = cursor.fetchone()
data_exist = row[0]     # 저장된 데이터의 유무(1 - 데이터 있음)

conn.close()

In [76]:
if data_exist == 1:      # 데이터 존재하면
    print(f'{month} 에 데이터가 존재합니다.')
else:
    conn = dbconnect.db_connect("DEMO_DW")
    cur = conn.cursor()

    for row in df.itertuples():
        sql = "insert into household_gu (통계년월, 시군구명, 행정동코드, 세대수) \
               values (%s, %s, %s, %s)"
        cur.execute(sql, (row[1], row[2], row[3], row[4]))

    conn.commit()
    print(f'{month} 데이터를 저장하였습니다.')
    conn.close()

202306 데이터를 저장하였습니다.
